In this notebook I would be making all the predictions for values in 'sales_pipeline.xlsx' file for which the 'Deal_Stage' is under progress.

At first I calculate the predictions for Deal_Stage whether it would be won or Not and then we go about predict future sales for 1 month, 1 Quarter, 1 Year

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sales = pd.read_excel('sales-pipeline.xlsx')
interactions=pd.read_excel('interactions.xlsx')

In [21]:
df = pd.read_csv('predictions.csv', index_col=0)
df.head()

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff,Ordered_Product,Ordered_Agent,Predictions_GBM,Predictions_RF
3,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,1,22,Won,Won
15,Umbrella Corporation,3ECHUPQY,Daniell Hammack,daniell@piedpiper.com,orpha@umbrellacorporation.com,In Progress,GTX Plus Basic,561,2016-01-17,2016-10-03,260,2,5,Won,Won
35,Ontomedia,1ZDDZD8B,Markita Hansen,markita@piedpiper.com,nedra@ontomedia.com,In Progress,MG Special,50,2016-02-13,2016-11-01,262,6,1,Won,Won
39,Hottechi,J53D8EOL,Kary Hendrixson,kary@piedpiper.com,juliette@hottechi.com,In Progress,GTX Plus Basic,601,2016-02-20,2016-05-30,100,2,9,Won,Won
42,Silis,NKPH3EFG,Cassey Cress,cassey@piedpiper.com,zulema@silis.com,In Progress,MG Advanced,560,2016-02-22,2016-09-16,207,1,10,Won,Won


In [22]:
df.drop(['Predictions_GBM', 'Predictions_RF'], inplace=True, axis=1)

In [23]:
print(min(df['Created Date']))
print(max(df['Close Date']))

2016-01-06
2019-12-31


In [24]:
import datetime
start = datetime.datetime(year=2016, 
    month=1,
    day=6)

end = datetime.datetime(year=2019, 
    month=12,
    day=31)

print(start, end)

2016-01-06 00:00:00 2019-12-31 00:00:00


In [25]:
text_df = interactions[(interactions['InteractionDate'] >= start) & (interactions['InteractionDate'] <= end)]

In [26]:
text_df.shape

(316279, 6)

In [27]:
df.shape

(2089, 13)

In [28]:
newdf = pd.merge(df, text_df, how='inner', left_on=['SalesAgentEmailID','ContactEmailID'], right_on = ['fromEmailId','toEmailId'])

In [29]:
newdf.shape

(167814, 19)

In [30]:
newdf.head()

,Account,Opportunity_ID,Sales_Agent,SalesAgentEmailID,ContactEmailID,Deal_Stage,Product,Close_Value,Created Date,Close Date,DateDiff,Ordered_Product,Ordered_Agent,interactionID,fromEmailId,toEmailId,InteractionDate,InteractionType(Call/Email/SMS),Extracted Interaction Text
0,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,1,22,4051152,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,2016-04-22,Email,I appreciate your contribution to Morgan Stanl...
1,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,1,22,4849530,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,2016-06-10,Email,I am pleased to accept your generous offer of ...
2,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,1,22,8752307,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,2016-06-26,Email,We assure you that you won't face any problems
3,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,1,22,8877493,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,2016-06-26,Call,There must have been a misunderstanding as we ...
4,Genco Pura Olive Oil Company,IBEUTCSW,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054,2016-01-06,2016-09-11,249,1,22,8832959,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,2016-07-16,Call,"On behalf of the rest of the management team, ..."


In [19]:
newdf['Deal_Stage'].unique()

array(['In Progress'], dtype=object)

In [37]:
cols_to_use = ['Close_Value', 'DateDiff','Extracted Interaction Text', 'Ordered_Agent','Ordered_Product']
newdf = newdf[cols_to_use]

In [38]:
newdf.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product
0,1054,249,I appreciate your contribution to Morgan Stanl...,22,1
1,1054,249,I am pleased to accept your generous offer of ...,22,1
2,1054,249,We assure you that you won't face any problems,22,1
3,1054,249,There must have been a misunderstanding as we ...,22,1
4,1054,249,"On behalf of the rest of the management team, ...",22,1


In [125]:
# Data cleaning

In [49]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
len(stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SIDDHARTH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [50]:
def cleaner(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", 'can not', phrase)
  
  # general
    phrase = re.sub(r"n\'t"," not", phrase)
    phrase = re.sub(r"\'re'"," are", phrase)
    phrase = re.sub(r"\'s"," is", phrase)
    phrase = re.sub(r"\'ll"," will", phrase)
    phrase = re.sub(r"\'d"," would", phrase)
    phrase = re.sub(r"\'t"," not", phrase)
    phrase = re.sub(r"\'ve"," have", phrase)
    phrase = re.sub(r"\'m"," am", phrase)
    
    return phrase

In [51]:
cleaned_title = []

for sentance in tqdm(newdf['Extracted Interaction Text'].values):
    sentance = str(sentance)
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = cleaner(sentance)
    sentance = re.sub(r'[?|!|\'|"|#|+]', r'', sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop)
    cleaned_title.append(sentance.strip())

100%|████████████████████████████████████████████████████████████████████████| 167814/167814 [01:17<00:00, 2179.20it/s]


In [52]:
newdf['Extracted Interaction Text'] = cleaned_title

In [53]:
newdf.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product
0,1054,249,appreciate contribution morgan stanley efforts...,22,1
1,1054,249,pleased accept generous offer engineering posi...,22,1
2,1054,249,assure face problems,22,1
3,1054,249,must misunderstanding already contracted anoth...,22,1
4,1054,249,behalf rest management team want tell much app...,22,1


In [54]:
model = pickle.load(open('text_model', 'rb'))
vect = pickle.load(open('vect_model', 'rb'))

In [55]:
Text_TFIDF = vect.transform(newdf['Extracted Interaction Text'])

In [56]:
test_agent=[]
for i in newdf['Ordered_Agent']:
    test_agent.append(i)
test_pdt=[]
for i in newdf['Ordered_Product']:
    test_pdt.append(i)
test_diff=[]
for i in newdf['DateDiff']:
    test_diff.append(i)
test_val=[]
for i in newdf['Close_Value']:
    test_val.append(i)

In [57]:
from scipy.sparse import hstack
Text_TFIDF_mix = (hstack((Text_TFIDF,np.array(test_agent)[:,None], np.array(test_pdt)[:,None], np.array(test_diff)[:,None], np.array(test_val)[:,None]))).tocsr()

In [58]:
Deal_pred = model.predict(Text_TFIDF_mix)

In [59]:
newdf['Deal_Closepred'] = Deal_pred

In [60]:
newdf.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product,Deal_Closepred
0,1054,249,appreciate contribution morgan stanley efforts...,22,1,1
1,1054,249,pleased accept generous offer engineering posi...,22,1,1
2,1054,249,assure face problems,22,1,1
3,1054,249,must misunderstanding already contracted anoth...,22,1,1
4,1054,249,behalf rest management team want tell much app...,22,1,1


In [61]:
# Predicting Close Values for sales prediction

In [62]:
model2 = pickle.load(open('gbm_model', 'rb'))
scaler = pickle.load(open('scaler_RS', 'rb'))

In [63]:
test = newdf[['Deal_Closepred','DateDiff', 'Ordered_Product','Ordered_Agent','Close_Value']]

In [64]:
test1 = scaler.transform(test[['Deal_Closepred','DateDiff', 'Ordered_Product','Ordered_Agent']])

In [65]:
close_pred = model2.predict(test1)

In [66]:
newdf['Close_pred'] = close_pred

In [67]:
newdf.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product,Deal_Closepred,Close_pred
0,1054,249,appreciate contribution morgan stanley efforts...,22,1,1,3380.947611
1,1054,249,pleased accept generous offer engineering posi...,22,1,1,3380.947611
2,1054,249,assure face problems,22,1,1,3380.947611
3,1054,249,must misunderstanding already contracted anoth...,22,1,1,3380.947611
4,1054,249,behalf rest management team want tell much app...,22,1,1,3380.947611


In [69]:
len(df)

2089

In [70]:
# Predicting future sales using the predictions we have generated

In [116]:
df1 = newdf.drop_duplicates(subset=['Close_Value', 'Ordered_Agent'], keep='last')

In [117]:
df1.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product,Deal_Closepred,Close_pred
47,1054,249,receiving letter proposal initially decided ac...,22,1,1,3380.947611
1561,2000,208,regret must cancel appointment discuss athleti...,28,1,1,3380.947611
2735,3180,695,sorry report mvc enterprises renewed contract ...,14,4,1,726.063073
2828,487,29,pleased inform accepted business proposal sent...,20,1,1,3360.046684
3170,1180,45,enjoyed presentation proposed change employee ...,16,4,1,705.162146


In [118]:
df1['Deal_Closepred'].value_counts()

1    686
0    149
Name: Deal_Closepred, dtype: int64

## 1. Sales for 1 month

In [71]:
# We shall predict for those values which have DateDiff <= 30

In [119]:
month = df1[df1['DateDiff'] <= 30]
month.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product,Deal_Closepred,Close_pred
2828,487,29,pleased inform accepted business proposal sent...,20,1,1,3360.046684
5929,560,12,please convey compliments entire staff exceedi...,9,4,1,705.162146
7819,1180,3,honored would choose fill vacancy must decline...,7,1,1,3391.625199
10228,493,23,read innovative plans improve tax deductible i...,16,4,1,705.162146
11311,1232,7,considered carefully must decline offer positi...,29,1,1,3360.046684


In [120]:
forecast_month = (sum(month[month['Deal_Closepred'] == 1]['Close_pred']))
print(forecast_month)

105352.10499169235


## 2. Prediction for Quarter (3 months)

In [121]:
quarter = df1[(df1['DateDiff'] <= 120)]
quarter.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product,Deal_Closepred,Close_pred
2828,487,29,pleased inform accepted business proposal sent...,20,1,1,3360.046684
3170,1180,45,enjoyed presentation proposed change employee ...,16,4,1,705.162146
5929,560,12,please convey compliments entire staff exceedi...,9,4,1,705.162146
7819,1180,3,honored would choose fill vacancy must decline...,7,1,1,3391.625199
10228,493,23,read innovative plans improve tax deductible i...,16,4,1,705.162146


In [122]:
forecast_quarter = (sum(quarter[quarter['Deal_Closepred'] == 1]['Close_pred']))
print(forecast_quarter)

326307.29196127545


## 3. Prediction for one Year

In [123]:
year = df1[(df1['DateDiff'] <= 365)]
year.head()

,Close_Value,DateDiff,Extracted Interaction Text,Ordered_Agent,Ordered_Product,Deal_Closepred,Close_pred
47,1054,249,receiving letter proposal initially decided ac...,22,1,1,3380.947611
1561,2000,208,regret must cancel appointment discuss athleti...,28,1,1,3380.947611
2828,487,29,pleased inform accepted business proposal sent...,20,1,1,3360.046684
3170,1180,45,enjoyed presentation proposed change employee ...,16,4,1,705.162146
4258,561,178,unfortunately home office unwilling consider n...,23,2,1,1214.940722


In [124]:
forecast_quarter = (sum(year[year['Deal_Closepred'] == 1]['Close_pred']))
print(forecast_quarter)

992618.7853325543


## Results

### 1. For all the values in the dataset, we predicted that 686 deals would be won and 149 deals would be lost.

##### 2. We then made the predictions of sales for 30 days, 60 days, 365 days

#### 1. For the first month we will make a predicted sales of Rs. 105352.10499

#### 2. For the first quarter we will make a predicted sales of Rs. 326307.29196

#### 3. For the entire year we make predicted sales of Rs. 992618.78533